In [1]:
import os

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    openai_api_key=os.environ.get('OPENAI_API_KEY'),
    openai_api_base=os.environ.get('CHATGPT_API_ENDPOINT')   
)

/Users/kariske/langchain/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from langchain.document_loaders import PyPDFLoader

In [3]:
loaders = [
    PyPDFLoader("docs/01.pdf"),
    PyPDFLoader("docs/02.pdf"),
    PyPDFLoader("docs/03.pdf"),
    PyPDFLoader("docs/04.pdf")
]

In [4]:
docs = []

for loader in loaders:
    docs.extend(loader.load())

In [5]:
print(docs)

[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '2024-01-31T12:28:37-08:00', 'moddate': '2024-01-31T12:39:58+08:00', 'title': 'Unknown', 'source': 'docs/01.pdf', 'total_pages': 23, 'page': 0, 'page_label': '1'}, page_content='衝\n一\n波\n$\x14\x19\x12\x0e$07&3\x0e\x13\x11\x13\x15\x0fJOEE\x01\x01\x01\x15$\x14\x19\x12\x0e$07&3\x0e\x13\x11\x13\x15\x0fJOEE\x01\x01\x01\x15 \x13\x11\x13\x15\x10\x12\x10\x14\x01\x01\x01ɨʹ\x01\x11\x16\x1b\x11\x17\x1b\x14\x11\x13\x11\x13\x15\x10\x12\x10\x14\x01\x01\x01ɨʹ\x01\x11\x16\x1b\x11\x17\x1b\x14\x11'), Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '2024-01-31T12:28:37-08:00', 'moddate': '2024-01-31T12:39:58+08:00', 'title': 'Unknown', 'source': 'docs/01.pdf', 'total_pages': 23, 'page': 1, 'page_label': '2'}, page_content='網路報導專區\n下載PDF手冊\n閱讀\n動態電子書\n23\u3000中\u3000環\t 蘭桂坊Bar Hopping\n25\u3000中\u3000環\t Argo\n26\u3000中\u3000環\t The Dispensary\n27\u3000上\u3000環\t Bar Leone\n28\u3000中\u3000環\t The Savor

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    separators=['\n\n', '\n', ' ', '']
)

splits = text_splitter.split_documents(docs)
len(splits)

109

In [8]:
from langchain_openai import OpenAIEmbeddings

In [9]:
embeddings = OpenAIEmbeddings(
    base_url=os.environ.get('CHATGPT_API_ENDPOINT'),
)

In [10]:
import chromadb

chromadb.api.client.SharedSystemClient.clear_system_cache()

In [11]:
from langchain.vectorstores import Chroma

persist_directory = './db'

In [12]:
!rm -rf ./db

In [13]:
vectordb = Chroma.from_documents(
    documents = splits,
    embedding = embeddings,
    persist_directory = persist_directory
)

print(vectordb._collection.count())

109


In [14]:
question = "有什麼西式美食推薦？"

docs_ss = vectordb.similarity_search(question, k=3)
docs_mmr = vectordb.max_marginal_relevance_search(question, k=2, fetch_k=3)

In [15]:
len(docs_ss), len(docs_mmr)

(3, 2)

In [16]:
docs_ss[0].page_content[:200]

'1312\n  挖掘在地好物\n慢活小區\n慢活小區\n 環球饗宴\n西貢擁有五花百門的美食，讓你品嚐到不\n同的異國料理，每一條街道各有味道，由\n街頭小吃到米其林星級體驗，應有盡有。\n褔民路有多家泰國餐廳，烹煮道地惹味的\n泰式料理；坐在西貢市場街和西貢海傍廣\n場一帶，邊喝手工咖啡邊看風景，悠閒自\n得；灣景街則匯聚了歐洲、日本和印尼的\n融合菜餐廳。環球美食以外，當然也有菠\n蘿油、車仔麵等港式美食，以及古早味的'

In [17]:
docs_mmr[0].page_content[:200]

'1312\n  挖掘在地好物\n慢活小區\n慢活小區\n 環球饗宴\n西貢擁有五花百門的美食，讓你品嚐到不\n同的異國料理，每一條街道各有味道，由\n街頭小吃到米其林星級體驗，應有盡有。\n褔民路有多家泰國餐廳，烹煮道地惹味的\n泰式料理；坐在西貢市場街和西貢海傍廣\n場一帶，邊喝手工咖啡邊看風景，悠閒自\n得；灣景街則匯聚了歐洲、日本和印尼的\n融合菜餐廳。環球美食以外，當然也有菠\n蘿油、車仔麵等港式美食，以及古早味的'

In [18]:
question = "有什麼景色優美的景點可以推薦？"

docs_ss = vectordb.similarity_search(
    question,
    k=3,
    filter={"source": "docs/03.pdf"}
)

for doc in docs_ss:
    print(doc.metadata)

{'creationdate': '2020-09-01T11:24:51+08:00', 'creator': 'Adobe InDesign 14.0 (Macintosh)', 'moddate': '2020-09-01T11:25:20+08:00', 'page': 8, 'page_label': '9', 'producer': 'Adobe PDF Library 15.0', 'source': 'docs/03.pdf', 'total_pages': 13, 'trapped': '/False'}
{'creationdate': '2020-09-01T11:24:51+08:00', 'creator': 'Adobe InDesign 14.0 (Macintosh)', 'moddate': '2020-09-01T11:25:20+08:00', 'page': 11, 'page_label': '12', 'producer': 'Adobe PDF Library 15.0', 'source': 'docs/03.pdf', 'total_pages': 13, 'trapped': '/False'}
{'creationdate': '2020-09-01T11:24:51+08:00', 'creator': 'Adobe InDesign 14.0 (Macintosh)', 'moddate': '2020-09-01T11:25:20+08:00', 'page': 9, 'page_label': '10', 'producer': 'Adobe PDF Library 15.0', 'source': 'docs/03.pdf', 'total_pages': 13, 'trapped': '/False'}


In [19]:
from langchain.llms import Ollama

In [20]:
chat = Ollama(model="openchat:latest")

/var/folders/1f/8yzdnxxs70zbqsthkv66wrvc0000gn/T/ipykernel_28425/3336436285.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  chat = Ollama(model="openchat:latest")


In [21]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [22]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="搜索的訊息來源於以下三個PDF檔案，分別是`docs/01.pdf`, `docs/03.pdf`, `docs/04.pdf`",
        type="string"
    ),
    AttributeInfo(
        name="page",
        description="信息來源的頁面",
        type="integer"
    )
]

In [23]:
document_content_description = "這裡存放是關於香港特色的旅遊勝地以及美食和特有文化紀錄"

retriever = SelfQueryRetriever.from_llm(
    llm=chat,
    vectorstore=vectordb,
    document_contents=document_content_description,
    metadata_field_info=metadata_field_info
)

In [24]:
question = "介紹一下香港特色美食？"

docs = retriever.invoke(question, k=5)

for doc in docs:
    print(doc.metadata)

{'creationdate': '2023-09-12T19:04:38+08:00', 'creator': 'Acrobat Pro 23.3.20244', 'moddate': '2023-10-26T16:52:25+00:00', 'page': 6, 'page_label': '7', 'producer': 'Acrobat Pro 23.3.20244', 'source': 'docs/04.pdf', 'title': '深水埗 ── 漫步指南', 'total_pages': 29}
{'creationdate': '2024-01-31T12:28:37-08:00', 'creator': 'PyPDF', 'moddate': '2024-01-31T12:39:58+08:00', 'page': 3, 'page_label': '4', 'producer': 'PyPDF', 'source': 'docs/01.pdf', 'title': 'Unknown', 'total_pages': 23}
{'creationdate': '2020-09-01T11:24:51+08:00', 'creator': 'Adobe InDesign 14.0 (Macintosh)', 'moddate': '2020-09-01T11:25:20+08:00', 'page': 7, 'page_label': '8', 'producer': 'Adobe PDF Library 15.0', 'source': 'docs/03.pdf', 'total_pages': 13, 'trapped': '/False'}
{'creationdate': '2023-09-12T19:04:38+08:00', 'creator': 'Acrobat Pro 23.3.20244', 'moddate': '2023-10-26T16:52:25+00:00', 'page': 7, 'page_label': '8', 'producer': 'Acrobat Pro 23.3.20244', 'source': 'docs/04.pdf', 'title': '深水埗 ── 漫步指南', 'total_pages': 

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [28]:
compressor = LLMChainExtractor.from_llm(chat)

compression_retriever = ContextualCompressionRetriever(
    base_retriever=vectordb.as_retriever(),
    base_compressor=compressor
)

In [30]:
question = "介紹西貢的景點"

compressed_docs = compression_retriever.invoke(question)

def pretty_print_docs(docs):
    print(
        f"\n\n{'-' * 60}".join([f"\n\n第{i+1}個景點: \n\n" + doc.page_content for i , doc in enumerate(docs)])
    )

pretty_print_docs(compressed_docs)



第1個景點: 

位於新界最東端的西貢，既沒有櫛比鱗次的摩天大廈，也不見五光十色的霓虹燈，有「香港後花園」之稱。

------------------------------------------------------------

第2個景點: 

西貢擁有五花百門的美食，讓你品嚐到不同的異國料理，每一條街道各有味道，由街頭小吃到米其林星級體驗，應有盡有。

------------------------------------------------------------

第3個景點: 

相片提供：TUGO CHENG
海灘及小島
感受腳底的柔軟白沙和清澈海水，環顧周圍趣怪
的野生動物，這樣的香港或許會讓你意想不到。
西貢隱身於香港僻靜的一隅，擁有純淨無污染的
海灘。沿著郊野公園遠足攀山，西貢的美景盡收
眼底；你也可以從西貢市中心的西貢公眾碼頭搭
乘街渡（即小型渡輪），闖進另一番天地。
外，西貢還有大大小小的島嶼，數不清的幽靜景
點俯拾皆是，絕對值得花上一天仔細探索。


In [31]:
compression_retriever = ContextualCompressionRetriever(
    base_retriever=vectordb.as_retriever(search_type="mmr"),
    base_compressor=compressor
)

In [32]:
question = "香港哪裡有最好吃的蛋塔？ 如果有，請提供該店的地址"

compressed_docs = compression_retriever.invoke(question)
pretty_print_docs(compressed_docs)



第1個景點: 

香港九龍城衙前圍道64號曉逸軒地下1號舖 電話：＋852-2383-8093
營業時間： 10 ： 00～23 ： 45 交通：港鐵宋皇臺站 B3出口步行5分鐘
恆發雞蛋仔
地址：香港九龍旺角通菜街139號地下 B 號舖
電話：＋852-2318-1996 營業時間： 24小時
交通：港鐵旺角站 B2出口步行3分鐘

------------------------------------------------------------

第2個景點: 

擁有超 過 6 0 年歷 史的 泰
昌 餅 家
中環擺花街35號
+852 8300 8301
www.taoheung.com.hk/tc/
brands/tai_cheong/
index.html

------------------------------------------------------------

第3個景點: 

地址：香港銅鑼灣告士打道310號27樓 （香港柏寧鉑爾曼酒店）
營業時間：（雞尾酒供應） 12 ： 00～凌晨00 ： 30
交通：港鐵銅鑼灣站 E 出口步行3分鐘
